In [1]:
# Imports
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
# Create connection, database, and collection within Mongo
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.atlstation_events_db
collection = db.event_list

In [3]:
# Set up chrome driver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/tatianafrattale/.wdm/drivers/chromedriver/mac64/93.0.4577.15/chromedriver] found in cache


In [4]:
# Url set up
url = "https://atlanticstation.com/events/list/"
request = requests.get(url)
browser.visit(url)

# Time sleep that allows for browser to fully load html
time.sleep(10)

In [5]:
# HTML parser
html = browser.html
soup = bs(html, "html.parser")

# Close browser session
browser.quit()

In [6]:
# Make empty lists for data
event_name = []
event_description = []
event_date = []
event_time = []
event_link= []

In [7]:
# Save all event cards to a variable
events = soup.find_all('div', class_='event-card')

In [8]:
# Loop through all events to pull necessary info
for event in events:
    # Find event name
    name = event.find('div', class_='event-card-title').text
    # Find event description
    descrip = event.find('div', class_='hover-card-body').text
    # Find date
    date1 = event.find('div', class_='event-card-date').text.strip()
    date = f'{date1}/21'
    # Find time
    time = event.find('div', class_='hover-card-content').text.split(" ")[8].strip()
    # Find link to event
    link = event.find('a', class_='full-link')['href']
    
    # Append data to empty lists
    event_name.append(name)
    event_description.append(descrip)
    event_date.append(date)
    event_time.append(time)
    event_link.append(link)
    
    # Create dictionary
    atl_events= {
        "event_name": name,
        "event_description": descrip,
        "event_date": date,
        "event_time": time,
        "event_link": link
    }
    
    # Insert data into collection in MongoDB
    collection.insert_one(atl_events)
    
    


In [9]:
# Create data frame
df = pd.DataFrame({
    "event_name": event_name,
    "event_description": event_description,
    "event_date": event_date,
    "event_time": event_time,
    "event_link": event_link
})
df.head(9)

,event_name,event_description,event_date,event_time,event_link
0,Elements of Style,"Check out our unique, personalized shopping an...",09/8/21,12:00,https://atlanticstation.com/event/elements-of-...
1,Black Bean Coffee Co. Pop-Up,"On Wednesdays, Thursdays and Fridays, Black Be...",09/9/21,8:00,https://atlanticstation.com/event/black-bean-c...
2,Black Bean Coffee Co. Pop-Up,"On Wednesdays, Thursdays and Fridays, Black Be...",09/10/21,8:00,https://atlanticstation.com/event/black-bean-c...
3,Dirty Birds Rally presented by Truist,Celebrate the start of the 2021 football seaso...,09/10/21,7:00,https://atlanticstation.com/event/dirty-birds-...
4,Maker’s Market,Shop local and celebrate Atlanta’s unstoppable...,09/11/21,11:00,https://atlanticstation.com/event/makers-marke...
5,Meditation and Mimosas,Enjoy a zen Sunday Funday with atl kula and At...,09/12/21,10:30,https://atlanticstation.com/event/meditation-a...
6,Tot Spot,Tot Spot returns to Atlantic Station with stor...,09/14/21,10:00,https://atlanticstation.com/event/tot-spot/202...
7,Wellness Wednesdays,Join Atlantic Station and atl kula for free ou...,09/15/21,6:30,https://atlanticstation.com/event/wellness-wed...
8,Black Bean Coffee Co. Pop-Up,"On Wednesdays, Thursdays and Fridays, Black Be...",09/16/21,8:00,https://atlanticstation.com/event/black-bean-c...
